# Run Predict

**Created on:** Monday, February 24, 2025, 08:36:14  
**Last modified:** February 2025
**Author:** anonymous author


[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/drive/12zrP3bqw8FftpLJu1uWQ2nvvBKH6iJ_2)

## Import Modules

In [1]:
import sys
if "google.colab" in sys.modules:  # Solo ejecuta en Colab
    !apt-get update -qq > /dev/null
    !apt-get install --reinstall --yes libcudnn9 libcudnn9-dev -qq > /dev/null
    !pip uninstall -y torch torchvision > /dev/null 2>&1
    !pip install torch torchvision --index-url https://download.pytorch.org/whl/cu124 > /dev/null 2>&1


W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
E: Package 'libcudnn9' has no installation candidate
E: Package 'libcudnn9-dev' has no installation candidate


In [2]:
import argparse
import sys
import os

import pandas as pd
import numpy as np
import cv2
from PIL import Image

import torch
import torch.nn as nn
import torchvision.models as models
import torchvision.transforms as transforms
from torch.utils.data import Dataset, DataLoader

try:
  import pytorch_lightning as pl
except:
  !pip install -q pytorch-lightning
  import pytorch_lightning as pl

from tqdm import tqdm

import timm
from IPython.display import display, HTML, Image as IPImage

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.0/823.0 kB 21.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 960.9/960.9 kB 48.3 MB/s eta 0:00:00


In [3]:
import torch
print("PyTorch Version:", torch.__version__)
print("CUDA Available:", torch.cuda.is_available())
print("CUDA Version:", torch.version.cuda)
!cat /usr/include/cudnn_version.h | grep CUDNN_MAJOR -A 2


PyTorch Version: 2.6.0+cu124
CUDA Available: True
CUDA Version: 12.4
#define CUDNN_MAJOR 9
#define CUDNN_MINOR 2
#define CUDNN_PATCHLEVEL 1
--
#define CUDNN_VERSION (CUDNN_MAJOR * 10000 + CUDNN_MINOR * 100 + CUDNN_PATCHLEVEL)

/* cannot use constexpr here since this is a C-only file */


In [4]:
import gdown
from typing import Dict

def download_weights(weights_urls: Dict[str, str], output_paths: Dict[str, str]) -> None:
    """
    Download model weights from Google Drive if they don't exist locally.

    Args:
        weights_urls (Dict[str, str]): Dictionary of model names and their Google Drive download URLs
        output_paths (Dict[str, str]): Dictionary of model names and their local save paths
    """
    for model_name, url in weights_urls.items():
        output_path = output_paths.get(model_name)

        if not output_path:
            print(f"No output path specified for {model_name}")
            continue

        try:
            if not os.path.exists(output_path):
                print(f"Downloading {model_name} weights...")
                gdown.download(url=url, output=output_path, quiet=False)
                print(f"Successfully downloaded {model_name} weights to {output_path}")
            else:
                print(f"Weights for {model_name} already exist at {output_path}")

        except Exception as e:
            print(f"Error downloading {model_name} weights: {e}")

def predict_to_dataframe(model, data_loader, df_to_save, device, column_name="organ_predict"):
    """
    Performs predictions using a model for all data in a loader and saves the results
    in a DataFrame.

    Args:
        model: PyTorch model to perform predictions
        data_loader: PyTorch DataLoader with input data
        df_to_save: Pandas DataFrame where results will be saved
        device: Device where inference will be executed (CPU or GPU)
        column_name: Name of the column where predictions will be saved

    Returns:
        DataFrame with saved predictions
    """
    model.eval()  # Ensure the model is in evaluation mode
    counter = 0

    # Iterate over all batches in the DataLoader with progress bar
    for batch in tqdm(data_loader, desc="Making predictions"):
        # If batch is a tuple or list, take only x
        if isinstance(batch, tuple) or isinstance(batch, list):
            x = batch[0]
        else:
            x = batch

        # Move data to the appropriate device
        x = x.to(device)

        # Perform inference without calculating gradients
        with torch.no_grad():
            # Get model predictions
            output = model(x)

            # Calculate predicted class
            y_pred = torch.argmax(output, dim=1)

            # Process each sample in the batch
            batch_size = x.size(0)
            for i in range(batch_size):
                # Save prediction in the DataFrame
                df_to_save.loc[counter, column_name] = y_pred[i].item()
                counter += 1

    return df_to_save

# Define transformations
transform = transforms.Compose([
    transforms.Resize((224, 224), interpolation=Image.LANCZOS),
    transforms.ToTensor(),
    transforms.Normalize([0.5990, 0.3664, 0.2769], [0.2847, 0.2190, 0.1772])
])
map_organ = {0:'pharynx',1:'esophagus',2:'stomach',3:'duodenum'}
map_stomach = {0:'A1',1:'L1',2:'P1',3:'G1',4:'A2',5:'L2',6:'P2',7:'G2',8:'A3',9:'L3',10:'P3',11:'G3',12:'A4',13:'L4',14:'P4',15:'G4',16:'A5',17:'L5',18:'P5',19:'A6',20:'L6',21:'P6', 22:'No quality'}

## Downloading Trained Weights



In [5]:
WEIGHTS_URLS = {
    'data_manager': "https://drive.google.com/uc?id=14ZmGnZs_Q-0pAcTPpRYItajAzZlnvvbQ",
    'trained_model_loader': "https://drive.google.com/uc?id=1eggzPzsYQSsrnGeAQlY52Pgo4ybMGkg1",
    'video': "https://drive.google.com/uc?id=1rmKKgonVazJxtDeOIXzUKW3y5Szx6hsV",
    'embedding': "https://drive.google.com/uc?id=1bAkdxB2_uR6oXYZNvxOSUAAHpHkSHSg1",
    'organ_model': "https://drive.google.com/uc?id=1pxcgpFmbIglPX99n-GPWrvh6lOP7T5l2",
    'stomach_model': "https://drive.google.com/uc?id=1PCVgzHVEb8UkB3FPMGt-qwHgNNj_TVrc"
}

OUTPUT_PATHS = {
    'data_manager': 'data_manager.py',
    'trained_model_loader': 'trained_model_loader.py',
    'video': 'video.mp4',
    'embedding': 'embedding.ckpt',
    'organ_model': 'organ_model.ckpt',
    'stomach_model': 'stomach_model.ckpt'
}

# Call the function to download weights
download_weights(WEIGHTS_URLS, OUTPUT_PATHS)

Downloading...
From (original): https://drive.google.com/uc?id=14ZmGnZs_Q-0pAcTPpRYItajAzZlnvvbQ
From (redirected): https://drive.google.com/uc?id=14ZmGnZs_Q-0pAcTPpRYItajAzZlnvvbQ&confirm=t&uuid=f18a728b-66ed-4382-a6a1-052c829294b5
To: /content/data_manager.py
100%|██████████| 7.24k/7.24k [00:00<00:00, 4.90MB/s]


Successfully downloaded data_manager weights to data_manager.py


Downloading...
From (original): https://drive.google.com/uc?id=1eggzPzsYQSsrnGeAQlY52Pgo4ybMGkg1
From (redirected): https://drive.google.com/uc?id=1eggzPzsYQSsrnGeAQlY52Pgo4ybMGkg1&confirm=t&uuid=7b3d0caa-b5a8-484c-a90a-9d47814be5d3
To: /content/trained_model_loader.py
100%|██████████| 4.76k/4.76k [00:00<00:00, 7.10MB/s]


Successfully downloaded trained_model_loader weights to trained_model_loader.py


Downloading...
From: https://drive.google.com/uc?id=1rmKKgonVazJxtDeOIXzUKW3y5Szx6hsV
To: /content/video.mp4
100%|██████████| 2.88M/2.88M [00:00<00:00, 92.5MB/s]


...

Successfully downloaded embedding weights to embedding.ckpt


Downloading...
From (original): https://drive.google.com/uc?id=1pxcgpFmbIglPX99n-GPWrvh6lOP7T5l2
From (redirected): https://drive.google.com/uc?id=1pxcgpFmbIglPX99n-GPWrvh6lOP7T5l2&confirm=t&uuid=4677c32f-0f4f-40b7-854d-b634765c1e38
To: /content/organ_model.ckpt
100%|██████████| 341M/341M [00:01<00:00, 233MB/s]


Successfully downloaded organ_model weights to organ_model.ckpt


Downloading...
From (original): https://drive.google.com/uc?id=1PCVgzHVEb8UkB3FPMGt-qwHgNNj_TVrc
From (redirected): https://drive.google.com/uc?id=1PCVgzHVEb8UkB3FPMGt-qwHgNNj_TVrc&confirm=t&uuid=3c011c39-d862-4868-a32c-a3410e8496f7
To: /content/stomach_model.ckpt
100%|██████████| 341M/341M [00:01<00:00, 262MB/s]

Successfully downloaded stomach_model weights to stomach_model.ckpt


In [6]:
# load modules
from data_manager import VideoFrameDataset, load_frames_from_video, get_sequences_space, expand_sequences, CustomDataset
from trained_model_loader import initialize_model, Identity, ModelInference, load_model_for_inference, load_vit_model_for_inference
%env PYDEVD_DISABLE_FILE_VALIDATION=1

env: PYDEVD_DISABLE_FILE_VALIDATION=1


In [7]:
#======================================
# Get and set all input parameters
#======================================
def get_args_parser():
    parser = argparse.ArgumentParser('Predict', add_help=False)
    # Video path
    parser.add_argument('--path_video', type=str, default=os.path.join("video.mp4"),
                        help='path image cnn embedding')
    # Model paths
    parser.add_argument('--path_image_embedding', type=str, default=os.path.join("embedding.ckpt"),
                        help='path image cnn embedding')
    parser.add_argument('--path_organ_classification', type=str, default=os.path.join("organ_model.ckpt"),
                        help='path organ transformer classification')
    parser.add_argument('--path_stomach_classification', type=str, default=os.path.join("stomach_model.ckpt"),
                        help='path stomach transformer classification')
    # Model parameters
    parser.add_argument("--num_workers", type=int, default=4, help="Number of workers in dataloader")
    parser.add_argument('--batch_size',type=int, default=16, help='Batch size per GPU (effective batch size is batch_size * accum_iter * # gpus') # 1024

    return parser

# Predict

In [8]:
if __name__ == '__main__':
    parser = get_args_parser()
    args, unknown = parser.parse_known_args()
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    if torch.cuda.is_available():
        torch.cuda.empty_cache()
    #==========================================
    # Embedding Representation
    #==========================================
    # Load the model
    # a) Load the backbone
    model_ft, CNN_family = initialize_model("convnext_tiny", 23, True, (224, 224))
    for param in model_ft.parameters():
        param.requires_grad = False
    # b) Load the weights
    model = load_model_for_inference(args.path_image_embedding, model_ft, device=device)
    # c) Preprocessing
    #### Add functions to crop frame: roi
    # d) get frame rate
    cap = cv2.VideoCapture(args.path_video)
    num_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    if not cap.isOpened():
        print(f"❌ Error: Could not open video {args.path_video}")
    frames_tensor = load_frames_from_video(cap, model, transform, device)
    cap.release()
    #==========================================
    # Dataloader
    #==========================================
    MAX_SEQ_LENGTH = 135 # temporal window trained
    df_setup_expanded = expand_sequences(num_frames,MAX_SEQ_LENGTH)
    assert num_frames == len(df_setup_expanded)
    # Define dataset
    dataset = CustomDataset(df_setup_expanded, frames_tensor)
    # Define DataLoader
    data_loader = DataLoader(dataset, batch_size=args.batch_size, shuffle=False, num_workers=args.num_workers)

    #####################################
    # 1) Organ Classification
    # ###################################
    # a) Cargar modelo para inferencia
    vit_model_organ = load_vit_model_for_inference(args.path_organ_classification, device, num_classes=4, max_seq_length=MAX_SEQ_LENGTH)
    print("Modelo ViT Organ cargado exitosamente para inferencia.")
    # b) Inferencia Organ
    df_setup_expanded["organ_predict"] = None
    df_setup_expanded = predict_to_dataframe(
        model=vit_model_organ,
        data_loader=data_loader,
        df_to_save=df_setup_expanded,
        device=device,
        column_name="organ_predict"
    )
    #####################################
    # 2) Stomach Site Classification
    #####################################
    # a) Cargar modelo para inferencia
    vit_model_organ = load_vit_model_for_inference(args.path_stomach_classification, device, num_classes=23, max_seq_length=MAX_SEQ_LENGTH)
    print("Modelo ViT cargado exitosamente para inferencia.")
    # b) Inferencia Organ
    df_setup_expanded["stomach_predict"] = None
    df_setup_expanded = predict_to_dataframe(
        model=vit_model_organ,
        data_loader=data_loader,
        df_to_save=df_setup_expanded,
        device=device,
        column_name="stomach_predict"
    )
    # map
    roi_index = df_setup_expanded[df_setup_expanded["organ_predict"] != 2].index
    df_setup_expanded.loc[roi_index,"stomach_predict"] = "No stomach"
    df_setup_expanded["organ_predict"]=df_setup_expanded["organ_predict"].replace(map_organ)
    df_setup_expanded["stomach_predict"]=df_setup_expanded["stomach_predict"].replace(map_stomach)


Downloading: "https://download.pytorch.org/models/convnext_tiny-983f1562.pth" to /root/.cache/torch/hub/checkpoints/convnext_tiny-983f1562.pth
100%|██████████| 109M/109M [00:00<00:00, 116MB/s]
Processing Frames: 100%|██████████| 152/152 [00:06<00:00, 25.24frame/s]
/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:624: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(


Modelo ViT Organ cargado exitosamente para inferencia.


Making predictions:   0%|          | 0/10 [00:00<?, ?it/s]/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:624: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(
Making predictions: 100%|██████████| 10/10 [00:01<00:00,  6.64it/s]


Modelo ViT cargado exitosamente para inferencia.


Making predictions:   0%|          | 0/10 [00:00<?, ?it/s]/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:624: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(
Making predictions: 100%|██████████| 10/10 [00:01<00:00,  6.59it/s]


In [9]:
# Open the video file
cap = cv2.VideoCapture(args.path_video)
frames_selected = []

# Current frame index
current_frame = 0

while True:
    # Read a frame from the video
    ret, frame = cap.read()
    # If no more frames, exit the loop
    if not ret:
        break
    frame = cv2.resize(frame, (224, 224))
    # Find the corresponding row in df_setup_expanded for the current frame
    frame_row = df_setup_expanded[df_setup_expanded['frame_roi'] == current_frame]
    if not frame_row.empty:
        # Get predictions
        organ_predict = frame_row['organ_predict'].values[0]
        stomach_predict = frame_row['stomach_predict'].values[0]
        # Configure text for predictions
        organ_text = f"Organ Pred.: {organ_predict}"
        stomach_text = f"Stomach Pred.: {stomach_predict}"
        # Get frame dimensions
        height, width = frame.shape[:2]
        # Add text in the central part of the frame
        cv2.putText(frame,organ_text,(0, height // 2),cv2.FONT_HERSHEY_SIMPLEX,0.5,(255, 255, 255),2)
        # Add text in the bottom part of the frame
        cv2.putText(frame, stomach_text, (0, height - 50),cv2.FONT_HERSHEY_SIMPLEX, 0.5,(255, 255, 255),2)
        # Convert frame to PIL Image and add to selected frames
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        frames_selected.append(Image.fromarray(frame))
    # Increment frame counter
    current_frame += 1
# Release video capture resources
cap.release()
# Define output GIF path
gif_path = 'output.gif'
# Save selected frames as an animated GIF
frames_selected[0].save(gif_path, save_all=True, append_images=frames_selected[1:], duration=100, loop=0)

In [10]:
# Display the GIF
print("Result")
display(IPImage(filename=gif_path))

Output hidden; open in https://colab.research.google.com to view.